# B0 Mapping - TSE

B0 mapping using a Cartesian TSE sequence where the echo is shifted to create a B0 dependency. 

### Imports

In [ ]:
import tempfile
from pathlib import Path

import matplotlib.pyplot as plt
import MRzeroCore as mr0
import numpy as np
from einops import rearrange
from mrpro.algorithms.reconstruction import DirectReconstruction
from mrpro.data import KData
from mrpro.data import SpatialDimension
from mrpro.data.traj_calculators import KTrajectoryCartesian

from mrseq.scripts.b0_tse import main as create_seq
from mrseq.utils import sys_defaults

### Settings
We are going to use a numerical phantom with a matrix size of 128 x 128.

In [ ]:
image_matrix_size = [128, 128]
repetition_time = 5
delta_te = np.array([0, 0.0004])

tmp = tempfile.TemporaryDirectory()
fname_mrd = Path(tmp.name) / 'b0_tse.mrd'

### Create the digital phantom

We use the standard Brainweb phantom from [MRzero](https://github.com/MRsources/MRzero-Core), but we choose the B1-field to be constant everywhere.

In [ ]:
phantom = mr0.util.load_phantom(image_matrix_size)
phantom.B1[:] = 1.0

### Create the B0 TSE sequence

To create the B0 TSE sequence, we use the previously imported [b0_tse script](../src/mrseq/scripts/b0_tse.py). 


In [ ]:
sequence, fname_seq = create_seq(
    system=sys_defaults,
    test_report=False,
    timing_check=False,
    fov_xy=float(phantom.size.numpy()[0]),
    tr=repetition_time,
    n_readout=image_matrix_size[0],
    n_phase_encoding=image_matrix_size[1],
    delta_te=delta_te,
    n_slice_encoding=1,
    n_echoes=4,
)

### Simulate the sequence
Now, we pass the sequence and the phantom to the MRzero simulation and save the simulated signal as an (ISMR)MRD file.

In [ ]:
mr0_sequence = mr0.Sequence.import_file(str(fname_seq.with_suffix('.seq')))
signal, ktraj_adc = mr0.util.simulate(mr0_sequence, phantom, accuracy=1e-5)
mr0.sig_to_mrd(fname_mrd, signal, sequence)

### Reconstruct the images at different inversion times

We use [MRpro](https://github.com/PTB-MR/MRpro) for the image reconstruction.

In [ ]:
kdata = KData.from_file(fname_mrd, trajectory=KTrajectoryCartesian())
kdata.header.encoding_matrix = SpatialDimension(z=1, y=image_matrix_size[1], x=2 * image_matrix_size[0])
kdata.header.recon_matrix = SpatialDimension(z=1, y=image_matrix_size[1], x=image_matrix_size[0])
recon = DirectReconstruction(kdata, csm=None)
idata = recon(kdata)

We can now plot the images at different inversion times.

In [ ]:
idat = idata.data.angle().numpy().squeeze()
fig, ax = plt.subplots(1, idat.shape[0], figsize=(3 * idata.shape[0], 3))
for i in range(idat.shape[0]):
    ax[i].imshow(idat[i, :, :], cmap='bwr', vmin=-np.pi, vmax=np.pi)
    ax[i].set_title(f'dTE = {np.round(delta_te[i] * 1e3, 2)} ms')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

### Estimate the B0 maps
We use a voxel-wise linear fit to estimate B0. Afterward, we compare them to the input and ensure they match.

In [ ]:
b0_input = np.roll(rearrange(phantom.B0.numpy().squeeze()[::-1, ::-1], 'x y -> y x'), shift=(1, 1), axis=(0, 1))

# Use proton density map as object mask
pd_input = np.roll(rearrange(phantom.PD.numpy().squeeze()[::-1, ::-1], 'x y -> y x'), shift=(1, 1), axis=(0, 1))
obj_mask = np.zeros_like(pd_input)
obj_mask[pd_input > 0] = 1

b0_measured = obj_mask * (idat[1] - idat[0]) / (2 * np.pi * (delta_te[1] - delta_te[0]))

fig, ax = plt.subplots(1, 3, figsize=(15, 3))
for cax in ax:
    cax.set_xticks([])
    cax.set_yticks([])

im = ax[0].imshow(b0_input, vmin=-50, vmax=50, cmap='bwr')
fig.colorbar(im, ax=ax[0], label='Input B0 (Hz)')

im = ax[1].imshow(b0_measured, vmin=-50, vmax=50, cmap='bwr')
fig.colorbar(im, ax=ax[1], label='Measured B0 (Hz)')

im = ax[2].imshow(b0_measured - b0_input, vmin=-5, vmax=5, cmap='bwr')
fig.colorbar(im, ax=ax[2], label='Difference B0 (Hz)')

relative_error = np.sum(np.abs(b0_input - b0_measured)) / np.sum(np.abs(b0_input))
print(f'Relative error {relative_error}')
assert relative_error < 0.15